### Przykładowe problemy związane ze skalowalnością zadań ML:

Ograniczenie CPU: Dane miesza sie w pamieci RAM, ale proces uczenia trwa za dlugo. Np. W przypadku koniecznosci sprawdzenia wielu kombinacji parametrow modelu, wielu modeli, itd. 


Ograniczenia pamieci: Dane sa na tyle duze ze nie mieszcza sie w pamieci RAM.


#### Pipeline

![](ml-Pipeline.png)

#### Pipeline Model

![](ml-PipelineModel.png)

### Potok przetwarzania ML

* <b>DataFrame</b>: ten interfejs API ML używa DataFrame ze Spark SQL jako zestawu danych ML, który może przechowywać różne typy danych. Na przykład DataFrame może mieć różne kolumny przechowujące tekst, wektory cech, prawdziwe etykiety i prognozy.


* <b>Transformer</b>: Transformator to algorytm, który może przekształcić jedną ramkę danych w inną ramkę danych. Na przykład model ML to transformator, który przekształca ramkę danych z funkcjami w ramkę danych z prognozami.


* <b>Estimator</b>: Estimator to algorytm, który można dopasować do DataFrame w celu wytworzenia transformatora. Np. Algorytm uczenia się jest estymatorem, który trenuje na DataFrame i tworzy model.


* <b>Pipeline</b>: Rurociąg łączy wiele transformatorów i estymatorów razem, aby określić przepływ pracy ML.


* <b>Parametr</b>: Wszystkie transformatory i estymatory mają teraz wspólny interfejs API do określania parametrów.

In [ ]:
import os
user_name = os.environ.get('USER')
print(user_name)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

In [ ]:
gs_path = f'gs://bdg-lab-{user_name}/survey/2020/survey_results_public.csv'

In [ ]:
db_name = user_name.replace('-','_')

In [ ]:
spark.sql(f'DROP DATABASE IF EXISTS {db_name} CASCADE')
spark.sql(f'CREATE DATABASE {db_name}')
spark.sql(f'USE {db_name}')

In [ ]:
table_name = "survey_2020"            

In [ ]:
spark.sql(f'DROP TABLE IF EXISTS {table_name}')

spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
          USING csv \
          OPTIONS (HEADER true, INFERSCHEMA true, NULLVALUE "NA") \
          LOCATION "{gs_path}"')

In [ ]:
spark.sql(f'describe {table_name}').show(100)

### Przygotowanie danych do analizy

W ramach zadania chcemy stworzyc klasyfikator, ktory bedzie przewidywac czy respondent zarabia wiecej niz 60000 USD rocznie

In [ ]:
spark_df= spark.sql(f'SELECT *, CAST((convertedComp > 60000) AS STRING) AS compAboveAvg \
                    FROM {table_name} where convertedComp IS NOT NULL ')
spark_df.limit(5).toPandas()

Dążymy do tego, żeby przygotować jeden wektor cech oraz jedną kolumnę z oznaczeniami. 

Pierwszy krok: feature extraction: kodujemy kolumny tekstowe na numeryczne, kodujemy wartosci liczbowe na reprezentacje onehotencoder. Nastepnie dokonujemy asemblacji do jednego wektora

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
# chcemy przewidziec compAboveAvg
y = 'compAboveAvg'
# na podstawie:
feature_columns = ['OpSys', 'EdLevel', 'MainBranch' , 'Country', 'JobSeek', 'YearsCode']

In [ ]:
#Zaczynamy od transformatora StringIndexer, zamieniajacego wartosci 'string' na liczbe

##### najpierw pokazujemy prosta petle z FOR, a potem zrefactorujmy do list comprehension

# dla cech, ktore zostana wykorzystane do predykcji

stringindexer_stages_1 = []
for c in feature_columns:
    stringindexer_stages_1.append (StringIndexer(inputCol=c, outputCol='strindexed_' + c).setHandleInvalid("keep"))


# i dla zmiennej objasnianej
stringindexer_stages_1.append(StringIndexer(inputCol=y, outputCol='label').setHandleInvalid("keep"))


In [ ]:
# Refactoring do list comprehension

stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c).setHandleInvalid("keep") for c in feature_columns]

# i dla zmiennej objasnianej
stringindexer_stages += [StringIndexer(inputCol=y, outputCol='label').setHandleInvalid("keep")]
stringindexer_stages

In [ ]:
# Po wykonaniu takiej transformacji do DF zostaje dodane  7 nowych kolumn z prefixem "strindexed_"
Pipeline(stages=stringindexer_stages).fit(spark_df).transform(spark_df).toPandas()

In [ ]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in feature_columns]

In [ ]:
# Rozbudowujemy pipeline..
#Po wykonaniu takiej transformacji do DF zostaje dodane  6 nowych kolumn z prefixem "onehot_". SparseV
pa = Pipeline(stages=stringindexer_stages + onehotencoder_stages).fit(spark_df).transform(spark_df).toPandas()

In [ ]:
pa.columns

In [ ]:
# Nowe kolumny zawieraja wartosci typu SparseVector zawierajacy mape bitowa.
pa['onehot_OpSys'].unique()

In [ ]:
# Polaczenie wszystkich kolumn predykcyjnych do jednej (features) ASEMBLACJA
extracted_columns = ['onehot_' + c for c in feature_columns]
vectorassembler_stage = VectorAssembler(inputCols=extracted_columns, outputCol='features') 

In [ ]:
# Polaczenie wszystkich krokow przygotowania danych w jednym potoku przetwarzania
final_columns = [y] + feature_columns + extracted_columns + ['features', 'label']

final_columns

In [ ]:
transformed_df = Pipeline(stages=stringindexer_stages + \
                          onehotencoder_stages + \
                          [vectorassembler_stage]).fit(spark_df).transform(spark_df).select(final_columns)

transformed_df.limit(5).toPandas()

### Podzial na zbior treningowy/testowy

In [ ]:
training, test = transformed_df.randomSplit([0.8, 0.2], seed=1234)

In [ ]:
training.count()

### Uczenie modelu - model.fit()

In [ ]:
# na poczatek wybierzemy drzewo decyzyjne. Nie musimy podawac zadnych parametrow
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')

In [ ]:
simple_model = Pipeline(stages=[dt]).fit(training)

In [ ]:
simple_model.stages[0]

### Predykcja - model.transform()

In [ ]:
pred_simple = simple_model.transform(test)

In [ ]:
show_columns = final_columns + ['prediction', 'rawPrediction', 'probability']
pred_simple.limit(5).select(show_columns).toPandas()

## Ewaluacje

In [ ]:
label_and_pred = pred_simple.select('label', 'prediction')
label_and_pred.groupBy('label', 'prediction').count().toPandas()

In [ ]:
# Ewaluator 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [ ]:
auroc_simple = evaluator.evaluate(pred_simple)
auroc_simple

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_m = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_m.evaluate(pred_simple)
accuracy

## Dodanie hiperparametrów 

In [ ]:
# Jake wartosci hiperparametru maxDepth maja byc przetwstowane
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(dt.maxDepth, [2,3,4,5,6]).\
    build()

In [ ]:
# Walidacja krzyrzowa wykonwyana w celu optymalizaji hiperparametrow
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

In [ ]:
# Budowa modelu na podstawie danych treningowych
cv_model = cv.fit(training)

In [ ]:
cv_model.bestModel

## Predykcja z nowym modelem

In [ ]:
# Jak wyglada predykcja na zbiorze danych treninigowych?
pred_cv = cv_model.transform(test)
show_columns = final_columns + ['prediction', 'rawPrediction', 'probability']
pred_cv.limit(5).select(show_columns).toPandas()

In [ ]:
# Confusion matrix
label_and_pred = pred_cv.select('label', 'prediction')
label_and_pred.groupBy('label', 'prediction').count().toPandas()

In [ ]:
auroc_cv = evaluator.evaluate(pred_cv)
auroc_cv

In [ ]:
acc_cv = evaluator_m.evaluate(pred_cv)
acc_cv

## Klasyfikacja za pomca Gradient Boosted Trees

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
model = gbt.fit(training)

In [ ]:
evaluator.evaluate(model.transform(test))

## Zadania:

* Czy mozna jeszcze poprawic jakosc predykcji: 
    * a) dodajac cechy
    * b) zmieniajac model
    * c) lepiej dobierajac parametry modelu ? 

In [ ]:
#Kod w R
#library(data.table)
#srv <- fread("survey_results_public.csv")
#srv$OpSys2 <- srv$OpSys == "Windows"
#library(rpart)
#srv$CompAboveAvg <- CompAboveAvg$ConvertedComp > 60e3
#dt_fit = rpart(CompAboveAvg ~ Age + EdLevel + JobSeek + OpSys + YearsCode , data = srv, method = 'class')
#pred_y = predict(dt_fit, type = 'class')
#table(predict(dt_fit, srv[,c("Age" , "EdLevel", "JobSeek", "OpSys", "YearsCode")], type = "class"), srv$CompAboveAvg)
#srv(cor)
